In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Kaggle https://www.kaggle.com/c/ieee-fraud-detection

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,Activation
from sklearn.model_selection import StratifiedKFold,cross_val_score
from keras.utils.generic_utils import get_custom_objects


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
train_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
train_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
sample_submission = pd.read_csv("/kaggle/input/ieee-fraud-detection/sample_submission.csv")
test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
train=train_transaction.merge(train_identity, left_on="TransactionID",right_on="TransactionID", how="left",suffixes=('_tran','_iden'))
test=test_transaction.merge(test_identity, left_on="TransactionID",right_on="TransactionID", how="left",suffixes=('_tran','_iden'))
print ("{} dataset has {} rows(samples) with {} columns(features) each.".format('train',*train.shape))  
print ("{} dataset has {} rows(samples) with {} columns(features) each.".format('test',*test.shape))  

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
scaler = MinMaxScaler()

Id = test['TransactionID']
train = train.drop('TransactionID',axis =1)
test = test.drop('TransactionID',axis =1)
y_train_d=train['isFraud']
X_train_d=train.drop(['isFraud'],axis=1)   


In [ ]:
train_n=pd.DataFrame(scaler.fit_transform(X_train_d),columns=X_train_d.columns,index=X_train_d.index)
test_n=pd.DataFrame(scaler.fit_transform(test),columns=test.columns,index=test.index)
X_train, X_validate, y_train, y_validate = train_test_split(train_n, y_train_d, test_size=0.2)

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
from keras.layers import LeakyReLU,Activation
from keras.callbacks import TensorBoard
import sys

from keras import callbacks
from keras import backend as K

def getBestModelfromTrials(trials):
    valid_trial_list = [trial for trial in trials
                            if STATUS_OK == trial['result']['status']]
    losses = [ float(trial['result']['loss']) for trial in valid_trial_list]
    index_having_minumum_loss = np.argmin(losses)
    best_trial_obj = valid_trial_list[index_having_minumum_loss]
    return best_trial_obj['result']['Trained_Model']

class MyLogger(callbacks.Callback):
    def __init__(self, n):
        self.n = n   # print loss & acc every n epochs

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.n == 0:
            curr_loss =logs.get('loss')
            curr_acc = logs.get('acc') * 100
            print("epoch = %4d  loss = %0.6f  acc = %0.2f%%" % (epoch, curr_loss, curr_acc))
    
    


def swish(x):
    return (K.sigmoid(x) * x)

my_logger = MyLogger(n=1)
#my_logger = TensorBoard(log_dir="logs/{}".format(time()))
#my_logger = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
#tensorboard --logdir output/Graph 
#class_weight = {0: 1., 1: 9.}
space = {'choice': hp.choice('num_layers',
                    [ {'layers':'two', },
                    {'layers':'three',
                    'units3': hp.uniform('units3', 64,1024), 
                    'dropout3': hp.uniform('dropout3', .25,.75)}
                    ]),

            'units1': hp.uniform('units1', 64,1024),
            'units2': hp.uniform('units2', 64,1024),

            'dropout1': hp.uniform('dropout1', .25,.75),
            'dropout2': hp.uniform('dropout2',  .25,.75),

            'batch_size' : hp.uniform('batch_size', 28,128),

            'nb_epochs' :  100,
            'optimizer': hp.choice('optimizer',['adam']),
            'activation': hp.choice('activation',['LeakyReLU'])
        }

def f_nn(params):   
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation
    from keras.optimizers import Adadelta, Adam, rmsprop

    print ('Params testing: ', params)
    #print("shape testing: {}, {}".format(params['choice']['units3']))
    model = Sequential()
   
    model.add(Dense(units=round(params['units1']), input_dim = X_train.shape[1], kernel_initializer = "glorot_uniform")) 
    if params['activation']== 'LeakyReLU':
        model.add(LeakyReLU()) 
    else:
        model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))

    model.add(Dense(units=round(params['units2']), kernel_initializer = "glorot_uniform")) 
    if params['activation']== 'LeakyReLU':
        model.add(LeakyReLU()) 
    else:
        model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout2']))

    if params['choice']['layers']== 'three':
        model.add(Dense(units=round(params['choice']['units3']), kernel_initializer = "glorot_uniform")) 
        if params['activation']== 'LeakyReLU':
            model.add(LeakyReLU()) 
        else:
            model.add(Activation(params['activation']))
        model.add(Dropout(params['choice']['dropout3']))    

    model.add(Dense(units=1, kernel_initializer='normal'))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=round(params['nb_epochs']), batch_size=round(params['batch_size']),verbose = 0,callbacks=[my_logger]
              #, class_weight=class_weight
             )

    pred_auc =model.predict_proba(X_validate, batch_size = 128, verbose = 0)
    acc = roc_auc_score(y_validate, pred_auc)
    print('AUC:', acc)
    sys.stdout.flush() 
    return {'loss': -acc, 'status': STATUS_OK}


def optimize(objective,space,evals, cores, trials, optimizer=tpe.suggest, random_state=0):
    
    best = fmin(objective, space, algo=tpe.suggest, max_evals=evals, trials = trials)
    trials = Trials()
    best_param = fmin(f_nn, space, algo=tpe.suggest, max_evals=10, trials=trials)
    best_model = getBestModelfromTrials(trials)
    return best_param,best_model
    

In [ ]:
trials = Trials()
cores = 32
n= 1000
start = time.time()
best_param,best_model = optimize(f_nn,space,evals = n, cores = cores,optimizer=tpe.suggest,trials = trials)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(n,end - start))

print ('best_model: ')
print (best_model)

In [ ]:
y_class=best_model.predict(test_n)
unscaled = scale_target.inverse_transform(pd.DataFrame(y_pred))
unscaled_df=pd.DataFrame(unscaled)
#unscaled_df.head()

subs_null = pd.DataFrame(index=testrows_with_null_value.index)
subs_null['id'] = testrows_with_null_value.index
subs_null['revenue'] = np.mean(unscaled)
#subs_null
## submission
subs = pd.DataFrame(index=test_n.index)
subs['id'] = test_n.index
subs['revenue'] = unscaled
subs_final = pd.concat([subs,subs_null], ignore_index=True)
subs_final.to_csv('submission.csv', index=False,float_format='%.0f')
subs_final.head()